In [11]:
import tensorflow as tf
import datetime, os
import keras

logs_base_dir="logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

UsageError: Line magic function `%tensorboard` not found.


In [12]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs2 (started 16:50:45 ago; pid 4940)
  - port 6006: logdir logs (started 17:32:55 ago; pid 9348)


In [13]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

path='data/APPENDED'
data_path='data'
ref_dataset= np.load(os.path.join(path,'refrigerator.dat.npy'))
print(ref_dataset[0])
ref_bins=4
ref_window_size=2401
input_shape=(2401, 1)

ref_dataset=ref_dataset[:,1:]
print(ref_dataset.shape)

bin_max_count=[0 for i in range(ref_bins)]
ref_clusters=[6.64953227,193.52627165,456.50688195]

[1.30313293e+09 2.22200000e+02 6.00000000e+00]
(1575991, 2)


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU

def load_cnn():
    
    model=Sequential()
    #kernel_regularizer=regularizers.l2(0.01)
    model.add(Conv1D(filters=8,kernel_size=5,input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
#    model.add(Conv1D(filters=32,kernel_size=3))
#    model.add(BatchNormalization())
#    model.add(Conv1D(filters=32,kernel_size=3))
#    model.add(BatchNormalization())
#    model.add(Conv1D(filters=32,kernel_size=3))
#    model.add(BatchNormalization())
#    model.add(Activation('relu'))
#    model.add(MaxPooling1D(pool_size=4))
#    model.add(Conv1D(filters=64,kernel_size=3,kernel_regularizer=regularizers.l2(0.01)))
#    model.add(BatchNormalization())
#    model.add(Conv1D(filters=64,kernel_size=3,kernel_regularizer=regularizers.l2(0.01)))
#    model.add(BatchNormalization())
#    model.add(Conv1D(filters=64,kernel_size=3,kernel_regularizer=regularizers.l2(0.01)))
#    model.add(BatchNormalization())
#    model.add(Activation('relu'))
#    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    #model.add(Dense(100,activation='relu'))
    model.add(Dropout(0.5))
#    model.add(Dense(256,activation='relu'))
#    model.add(Dense(128,activation='relu'))
#    model.add(Dense(64,activation='relu'))
#    model.add(Dense(32,activation='relu'))
    model.add(Dense(3,activation='sigmoid'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 1e-6
    momentum = 0.8
    opt= Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
import collections
from keras.utils import np_utils
from matplotlib import pyplot as plt
from keras.utils import normalize
import csv


bin_dict=collections.Counter(ref_dataset[:,1])

ref_data=[]
ref_target=[]

model=load_cnn()


k=0
count=0
graph_no=0
key_min = min(bin_dict.keys(), key=(lambda k: bin_dict[k]))
ref_min_bin=bin_dict[key_min]
print(ref_min_bin)

epo=0
while(epo<20):
    graph_no=0
    for i in range(ref_dataset.shape[0]-ref_window_size-1-200000):
        ref_data.append(ref_dataset[i:i+ref_window_size,0])

        appliance=ref_dataset[i+ref_window_size-1,1]
        #print(appliance)

        total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
        prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
        prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
        prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
        #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
        #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))
        ref_target.append([prob0,prob1,prob2])
        k=k+1
        if(k==250000):
            data=np.array(ref_data)/500.0
            data = normalize(data, axis=1)
            target=np.array(ref_target)
            data=data.reshape(data.shape[0],data.shape[1],1)
            #print(target.shape,data[0],target[0])

            if(count!=10):
                logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
                tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                mc = keras.callbacks.ModelCheckpoint('weights/weights{epoch:08d}.h5', save_weights_only=True)
                history=model.fit(data,target,epochs=10,callbacks=[mc]) 
                
                for layer in model.layers:
                    weights = layer.get_weights()
                    print(weights)
                ref_data=[]
                ref_target=[]
                k=0
                count=count+1

            else:
    #            ref_data=[]
    #            ref_target=[]        
    #            for i in range(10000):
    #                ref_data.append(ref_dataset[700000+i:700000+i+ref_window_size,0])

    #                appliance=ref_dataset[700000+i+ref_window_size-1,1]
                    #print(appliance)

    #                total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
    #                prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
    #                prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
    #                prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
                    #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
                    #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))

    #                ref_target.append([prob0,prob1,prob2])

    #            data=np.array(ref_data)/500.0
    #            data = normalize(data, axis=1)
    #            target=np.array(ref_target)
    #            data=data.reshape(data.shape[0],data.shape[1],1)

                
                print(model.evaluate(data,target),epo,graph_no)
                results=model.predict(data)
                plt.plot(np.argmax(target,axis=1),'b',label='ACTUAL')
                plt.plot(np.argmax(results,axis=1),'r--',label='PREDICTED')
                plt.legend()
                plt.savefig('graphs_soft_association/'+str(epo)+str(graph_no)+'.jpg')
                plt.close()

                ref_data=[]
                ref_target=[]
                k=0
                count=0
                graph_no=graph_no+1
                
                #print(target.shape)
                #print(results.shape)
                actual=np.argmax(target,axis=1)
                predicted=np.argmax(results,axis=1)
                
                tp=actual[predicted>0]
                tp_count=len(np.where(tp>0)[0])
                fp_count=len(np.where(tp==0)[0])
                
                tn=actual[predicted==0]
                tn_count=len(np.where(tn==0)[0])
                fn_count=len(np.where(tn>0)[0])
                
                p=len(np.where(actual>0)[0])
                n=len(np.where(actual==0)[0])
                try:
                    recall=tp_count/(tp_count+fn_count)*100
                    precision=tp_count/(tp_count+fp_count)*100
                    accuracy=(tp_count+tn_count)/(p+n)*100
                    f1=2*(precision*recall)/(precision+recall)
                
                    print("==========================window "+str(i)+"-"+str(i+ref_window_size)+"==========================")
                    print("TP:",tp_count,"FP:",fp_count,"TN:",tn_count,"FN:",fn_count,"P:",p,"N:",n)
                    print('Recall:',recall,"Precision:",precision,"Accuaracy:",accuracy,"F1:",f1)
                except:
                    pass
    epo=epo+1

3
Epoch 1/10
250000/250000 [==============================] - 570s 2ms/step - loss: 0.0952 - accuracy: 0.9643
Epoch 2/10
250000/250000 [==============================] - 565s 2ms/step - loss: 0.0508 - accuracy: 0.9837
Epoch 3/10
250000/250000 [==============================] - 567s 2ms/step - loss: 0.0395 - accuracy: 0.9886
Epoch 4/10
250000/250000 [==============================] - 562s 2ms/step - loss: 0.0348 - accuracy: 0.9909
Epoch 5/10
 90080/250000 [=========>....................] - ETA: 6:07 - loss: 0.0325 - accuracy: 0.9919

In [ ]:
np.argmax(test_target,axis=1).shape

In [ ]:
test_target.shape

In [ ]:
test_target

In [ ]:
np.argmax(test_target,axis=1)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()
    print(weights)